In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc("font",size=14)
sns.set(style="white")
sns.set(style="whitegrid",color_codes=True)
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import pyodbc 

In [13]:
import pyhdb
conn = pyhdb.connect(
    host="cchgpasdb2",
    port=32553,
    user="MART_RDR_2",
    password="Re@d3r13"
)

In [14]:
cursor = conn.cursor()
cursor.execute("select current_date from dummy")
for row in cursor.fetchall():
    print(row)

(datetime.date(2018, 2, 19),)


# Customer Claims Hana Connection


In [15]:
df = pd.read_sql_query("""select 
	Claim.CLAIM_NUM 
	,Claim.INVENTORY_STYLE_CD 
	,Claim.INVENTORY_SIZE_CD 
	,Claim.INVENTORY_BACKING_CD 
	,Claim.INVENTORY_COLOR_CD 
	,Claim.DIM_DIVISION_CUSTOMER_GK 
	,Claim.F1ROLL
	,Claim.F1PROL
	,Claim.F1GROL
	,Claim.F1DLOT
	,Claim.Claim_Status_Type_Cd
	,sum(claim) as Claim 
	,sum(Total_Claim) as Total_Claim 
	,sum(Claim_Cnt) as Total_Claim_Cnt 
	,sum(Trx_cnt) as Trx_cnt 
	,sum(Net_Sales) as Net_Sales  
	from  
		(  
			select 
					CLAIM_NUM 
					,INVENTORY_STYLE_CD 
					,INVENTORY_SIZE_CD 
					,INVENTORY_BACKING_CD 
					,INVENTORY_COLOR_CD 
					,DIM_DIVISION_CUSTOMER_GK 
					,F1ROLL
					,F1PROL
					,F1GROL
					,F1DLOT
					,Claim_Status_Type_Cd
					,sum(TOTAL_CLAIMS_DOLLAR_AMT) as Claim 
			from 
					"MHK_EDW_MART"."FACT_CUSTOMER_CLAIMS" A 
					,"MHK_EDW_MART"."DIM_CLAIM_STATUS" B 
					,"MHK_EDW_MART"."DIM_CLAIM_REASON" C 
					,"MHK_EDW_MART"."DIM_FINISHED_PRODUCT_MFG_TYPE1" D 
					,"MHK_EDW_MART"."FIP010"  E
			where 
					A.DIM_CLAIM_STATUS_PK=B.DIM_CLAIM_STATUS_PK 
					and A.DIM_CLAIM_REASON_PK = C.DIM_CLAIM_REASON_PK 
					and A.DIM_FINISHED_PRODUCT_MFG_GK = D.DIM_FINISHED_PRODUCT_MFG_GK 
					and QUALITY_RELATED_CD = 'Y' and A.DIM_FINISHED_PRODUCT_MFG_GK >0 
					and A.FINISHED_ROLL_NUM = E.F1ROLL
					and Claim_Status_Type_Cd in ('48','49','50','51','30','35','36','34')
					and A.CLAIM_STATUS_DATE_PK >20170101
			group by 
					CLAIM_NUM 
					,INVENTORY_STYLE_CD 
					,INVENTORY_SIZE_CD 
					,INVENTORY_BACKING_CD 
					,INVENTORY_COLOR_CD 
					,DIM_DIVISION_CUSTOMER_GK 
					,F1ROLL
					,F1PROL
					,F1GROL
					,F1DLOT
					,Claim_Status_Type_Cd
			having 
					sum(TOTAL_CLAIMS_DOLLAR_AMT) >0 
		) Claim ,
		( 
				select 
						INVENTORY_STYLE_CD 
						,INVENTORY_SIZE_CD 
						,INVENTORY_BACKING_CD 
						,INVENTORY_COLOR_CD 
						,DIM_DIVISION_CUSTOMER_GK 
						,sum(TOTAL_CLAIMS_DOLLAR_AMT) as Total_Claim 
						,count(distinct CLAIM_NUM) as Claim_Cnt 
				from 
						"MHK_EDW_MART"."FACT_CUSTOMER_CLAIMS" A 
						,"MHK_EDW_MART"."DIM_CLAIM_STATUS" B 
						,"MHK_EDW_MART"."DIM_CLAIM_REASON" C 
						,"MHK_EDW_MART"."DIM_FINISHED_PRODUCT_MFG_TYPE1" D 
						,"MHK_EDW_MART"."DIM_BUSINESS_UNIT_DATE" E 
						,"MHK_EDW_MART"."FIP010"  F
						
				where 
						A.DIM_CLAIM_STATUS_PK=B.DIM_CLAIM_STATUS_PK 
						and A.DIM_CLAIM_REASON_PK = C.DIM_CLAIM_REASON_PK 
						and A.DIM_FINISHED_PRODUCT_MFG_GK = D.DIM_FINISHED_PRODUCT_MFG_GK 
						and A.FINISHED_ROLL_NUM = F.F1ROLL
						and QUALITY_RELATED_CD = 'Y' 
						and Claim_Status_Type_Cd in ('48','49','50','51','30','35','36','34') 
						and A.CLAIM_STATUS_DATE_PK =E.DIM_DATE_PK 
						and BUSINESS_UNIT_CD='MF' 
						and FIRST_OF_FISCAL_YEAR_DT between '2016-01-01' and Current_date 
						and A.DIM_FINISHED_PRODUCT_MFG_GK >0 
						
				group by 
					INVENTORY_STYLE_CD 
					,INVENTORY_SIZE_CD 
					,INVENTORY_BACKING_CD 
					,INVENTORY_COLOR_CD 
					,DIM_DIVISION_CUSTOMER_GK
				
				having sum(TOTAL_CLAIMS_DOLLAR_AMT) >0  )HClaim , 
				( 
						select 
								INVENTORY_STYLE_CD 
								,INVENTORY_SIZE_CD 
								,INVENTORY_BACKING_CD 
								,INVENTORY_COLOR_CD 
								,SOLD_TO_CUSTOMER_GK 
								,Count(distinct to_char(SOLD_TO_CUSTOMER_GK) || to_char(DOCUMENT_NUM)) as Trx_cnt 
								,sum(Net_Sales_amt) as Net_Sales 
					  from "MHK_EDW_MART"."FACT_CAMS_GL_SALES_INTERFACE" A 
					  		,"MHK_EDW_MART"."DIM_FINISHED_PRODUCT_MFG_TYPE1" B 
					  		,"MHK_EDW_MART"."DIM_BUSINESS_UNIT_DATE" C  
					  where 
					  		A.ENTRY_DATE_PK =C.DIM_DATE_PK 
					  		and C.BUSINESS_UNIT_CD='MF' 
					  		and FIRST_OF_FISCAL_YEAR_DT 
					  		between '2016-01-01' and Current_date 
					  		and A.FINISHED_PRODUCT_MFG_GK = B.DIM_FINISHED_PRODUCT_MFG_GK 
					  group by 
					  		INVENTORY_STYLE_CD 
					  		,INVENTORY_SIZE_CD 
					  		,INVENTORY_BACKING_CD 
					  		,INVENTORY_COLOR_CD 
					  		,SOLD_TO_CUSTOMER_GK 
					  having sum(Net_Sales_amt)>0 )HSales  
					  where 
					  		Claim.INVENTORY_STYLE_CD = HClaim.INVENTORY_STYLE_CD 
					  		and Claim.INVENTORY_SIZE_CD = HClaim.INVENTORY_SIZE_CD 
					  		and Claim.INVENTORY_BACKING_CD = HClaim.INVENTORY_BACKING_CD 
					  		and Claim.INVENTORY_COLOR_CD = HClaim.INVENTORY_COLOR_CD 
					  		and Claim.DIM_DIVISION_CUSTOMER_GK = HClaim.DIM_DIVISION_CUSTOMER_GK
					  		and	Claim.INVENTORY_STYLE_CD = HSales.INVENTORY_STYLE_CD 
					  		and Claim.INVENTORY_SIZE_CD = HSales.INVENTORY_SIZE_CD 
					  		and Claim.INVENTORY_BACKING_CD = HSales.INVENTORY_BACKING_CD 
					  		and Claim.INVENTORY_COLOR_CD = HSales.INVENTORY_COLOR_CD 
					  		and Claim.DIM_DIVISION_CUSTOMER_GK = HSales.SOLD_TO_CUSTOMER_GK 
					  		 
					 group by 
					 		Claim.CLAIM_NUM 
					 		,Claim.INVENTORY_STYLE_CD 
					 		,Claim.INVENTORY_SIZE_CD 
					 		,Claim.INVENTORY_BACKING_CD 
					 		,Claim.INVENTORY_COLOR_CD 
					 		,Claim.DIM_DIVISION_CUSTOMER_GK
					 		,Claim.F1ROLL
							,Claim.F1PROL
							,Claim.F1GROL
							,Claim.F1DLOT
							,Claim.Claim_Status_Type_Cd;
							
						
""",conn)
conn.close()
#df.head(10)
print(df.shape)
print(list(df.columns))



(29980, 16)
['CLAIM_NUM', 'INVENTORY_STYLE_CD', 'INVENTORY_SIZE_CD', 'INVENTORY_BACKING_CD', 'INVENTORY_COLOR_CD', 'DIM_DIVISION_CUSTOMER_GK', 'F1ROLL', 'F1PROL', 'F1GROL', 'F1DLOT', 'CLAIM_STATUS_TYPE_CD', 'CLAIM', 'TOTAL_CLAIM', 'TOTAL_CLAIM_CNT', 'TRX_CNT', 'NET_SALES']


In [16]:
df.to_csv('claim_product2017.csv')

In [10]:
df.head(20)

,CLAIM_NUM,INVENTORY_STYLE_CD,INVENTORY_SIZE_CD,INVENTORY_BACKING_CD,INVENTORY_COLOR_CD,DIM_DIVISION_CUSTOMER_GK,F1ROLL,F1PROL,F1GROL,F1DLOT,CLAIM_STATUS_TYPE_CD,CLAIM,TOTAL_CLAIM,TOTAL_CLAIM_CNT,TRX_CNT,NET_SALES
0,6970910,28172,1200,A,757,238677,04931960,,4765299,P35166,50,119.26,119.26,1,92,691740.56
1,7137990,27861,1200,A,752,244199,10520749,,5261321,864382$,49,1096.19,1096.19,1,96,70697.32
2,6984793,P67,0600,PB,716,236744,99439447,,,PAD,50,37.80,264.60,4,85,118985.00
3,6984793,P67,0600,PB,716,236744,99439449,,,PAD,50,37.80,264.60,4,85,118985.00
4,7069471,P67,0600,PB,716,236744,09265176,,,PAD,50,37.80,264.60,4,85,118985.00
5,6984793,P67,0600,PB,716,236744,99439448,,,PAD,50,37.80,264.60,4,85,118985.00
6,6984793,P67,0600,PB,716,236744,99439450,,,PAD,50,37.80,264.60,4,85,118985.00
7,7137929,P67,0600,PB,716,236744,10009380,,,PAD,50,37.80,264.60,4,85,118985.00
8,7025143,28072,1200,A,717,235416,07155511,,4979206,A40981$,50,126.50,126.50,1,13,191995.21
9,7023549,PW81,0600,PB,81,258325,93828158,,,PAD,30,50.40,176.40,1,27,23823.00
